In [1]:
# -*- coding: utf-8 -*-

"""
TFI Trotter Circuit Analysis with IBM Eagle Topology

This example demonstrates how to create a TFI (Transverse Field Ising) Trotter circuit
using the IBM Eagle topology with 127 qubits and analyze it with Pauli propagation.
"""

import numpy as np
from qiskit.quantum_info import Pauli, Statevector

# Custom package imports  
from pauli_propagation import PauliTerm, PauliPropagator
from pauli_propagation.utils import encode_pauli
from pauli_propagation.circuit_topologies import tfi_trotter_circuit


In [2]:
# nq = 127  # IBM Eagle has 127 qubits
# nl = 10    # 5 layers

# qc_tfi = tfi_trotter_circuit(nq, nl)

# pauli_label = "I" * 62 + "Z" + "I" * (nq - 63)  # 62 I's + 1 Z + 64 I's = 127 total
# key = encode_pauli(Pauli(pauli_label))
# init_term = PauliTerm(1.0, key, nq)

# product_label = "0" * nq

# prop_tfi = PauliPropagator(qc_tfi)

# max_weight = 4

# layers_tfi = prop_tfi.propagate(init_term, max_weight=max_weight, use_parallel=False)

# exp_tfi = prop_tfi.expectation_pauli_sum(layers_tfi[-1], product_label)

# print(exp_tfi)

In [3]:
# len(layers_tfi[-1])

In [ ]:
k = 7       # 你想要的子系统 qubit 数，比如 8、10、12、……
n_qubits_small = 7

# 整个 IBM Eagle 127-qubit 的 1-based 拓扑列表
ibm_eagle_topology = [(1, 2), (1, 15), (2, 3), (3, 4), (4, 5), (5, 6), (5, 16),(6, 7), (7, 8), (8, 9), (9, 10), (9, 17), (10, 11), 
                      (11, 12),(12, 13), (13, 14), (13, 18), (15, 19), (16, 23), (17, 27),(18, 31), (19, 20), (20, 21), (21, 22), 
                      (21, 34), (22, 23),(23, 24), (24, 25), (25, 26), (25, 35), (26, 27), (27, 28),(28, 29), (29, 30), (29, 36), 
                      (30, 31), (31, 32), (32, 33),(33, 37), (34, 40), (35, 44), (36, 48), (37, 52), (38, 39),(38, 53), (39, 40), 
                      (40, 41), (41, 42), (42, 43), (42, 54),(43, 44), (44, 45), (45, 46), (46, 47), (46, 55), (47, 48),(48, 49), 
                      (49, 50), (50, 51), (50, 56), (51, 52), (53, 57),(54, 61), (55, 65), (56, 69), (57, 58), (58, 59), (59, 60),
                      (59, 72), (60, 61), (61, 62), (62, 63), (63, 64), (63, 73),(64, 65), (65, 66), (66, 67), (67, 68), (67, 74), 
                      (68, 69),(69, 70), (70, 71), (71, 75), (72, 78), (73, 82), (74, 86),(75, 90), (76, 77), (76, 91), (77, 78), 
                      (78, 79), (79, 80),(80, 81), (80, 92), (81, 82), (82, 83), (83, 84), (84, 85),(84, 93), (85, 86), (86, 87), 
                      (87, 88), (88, 89), (88, 94),(89, 90), (91, 95), (92, 99), (93, 103), (94, 107), (95, 96),(96, 97), (97, 98), 
                      (97, 110), (98, 99), (99, 100), (100, 101),(101, 102), (101, 111), (102, 103), (103, 104), (104, 105),
                      (105, 106),(105, 112), (106, 107), (107, 108), (108, 109),(109, 113), (110, 115), (111, 119), (112, 123), 
                      (113, 127),(114, 115), (115, 116), (116, 117), (117, 118), (118, 119),(119, 120), (120, 121), (121, 122), 
                      (122, 123),(123, 124),(124, 125), (125, 126), (126, 127)]

subgraph_edges = [(u, v) for (u, v) in ibm_eagle_topology if u <= k and v <= k]

n_layers_small = 10         # Trotter 层数，可以自己修改
theta_h_small = 0.7        # RX 旋转角度，自己调整
rzz_theta_small = np.pi/4  # RZZ 旋转角度

qc_small = tfi_trotter_circuit(n_qubits_small, n_layers_small, subgraph_edges,
                               start_with_ZZ=True,
                               rx_theta=theta_h_small,
                               rzz_theta=rzz_theta_small)

p = 5
pauli_label_pauli = "I" * (p - 1) + "Z" + "I" * (n_qubits_small - p)
# 例子：如果 k=10 且 p=3，pauli_label_pauli 就是 "IIZIIIIIII"
key = encode_pauli(Pauli(pauli_label_pauli))
init_term = PauliTerm(1.0, key, n_qubits_small)

prop = PauliPropagator(qc_small)
layers = prop.propagate(init_term, max_weight=None, use_parallel=False)
product_label = "0" * n_qubits_small  # "0" 表示按 Z basis 测量
exp_pauli = prop.expectation_pauli_sum(layers[-1], product_label)
print('pauli propagation expectation value: ', exp_pauli)

state_lbl = "0" * n_qubits_small
sv = Statevector.from_label(state_lbl)

sv_after = sv.evolve(qc_small)

pauli_lbl_state = "I" * (p - 1) + "Z" + "I" * (n_qubits_small - p)
pauli_op = Pauli(pauli_lbl_state)

exp_state = sv_after.expectation_value(pauli_op).real

print("\n=== 对比结果 ===")
print(f"Pauli Propagation 得到： {exp_pauli:.6f}")
print(f"Statevector 得到：      {exp_state:.6f}")
print(f"差值：               {abs(exp_pauli - exp_state):.2e}")

# weight = 7, truth: 

Propagating, max weight: None:   0%|          | 0/130 [00:00<?, ?it/s]

pauli propagation expectation value:  0.06098859236115396

=== 对比结果 ===
Pauli Propagation 得到： 0.060989
Statevector 得到：      0.060989
差值：               5.97e-16
